# A <strong>hypergeometric</strong> question

In my small department at UiB, consisting of 20 people, we are actually two named *Hans Martin*. After several confusing situations, Halvor said agitated: "What are the odds that there are two in this room with that name?"

The department's logician, Wim, quickly sketched out a solution from first principles, arriving at something resembling the hypergeometric distribution. Despite his effort, no one in the quite educated group was able to give an answer.

(According to https://www.ssb.no/befolkning/navn/statistikk/navn there are 700 with that name, assume a male population of 2.5million)

### Solution (part 1) - First thought: let's just simulate it

In [30]:
import math
import random
from collections import Counter

pop = [0]*2500000  # assume a population of 2.5million
pop[:700] = [1]*700 # By SSB, about 700 with this name
random.shuffle(pop)

num_specials = []

# run 10 million experiments
for i in range(10000000): 
    
    num_specials.append(sum(random.choices(pop,k=20))) # pick 20 random

prob = Counter(num_specials)[2] / len(num_specials) # probability of (exactly) 2
print('Number of 2 occurrences: '+str(Counter(num_specials)[2]))
print('The chance is about 1/' + str(round(1/prob)))

Number of 2 occurrences: 143
The chance is about 1/69930


### Answer $ \approx \frac{1}{70\,000} $

Alright, so it happened <strong>143</strong> times out of 10 million simulations. The chance is $ \frac{1}{69\,930} \approx 0.0014$%. That's pretty unlikely!

### Solution (part 2) - Verify the simulation using Wim's intuition:

Given the question, a valid selection, requiring $|s|=2$ special elements and $|o|=18$ other elements is
$\overbrace{[s_{1}, s_{2}, o_{1},o_{2},... ,o_{18}]}^{n=20} $.

We disregard ordering, since we don't care *where* they sit in the office, only that $2$ special elements are in the $20$. This means to not count a selection if *all* the elements in it has been selected before (in a different order), called <strong>combinations without replacement</strong>.

A gentle start is to count all 2-combinations of 20. First we count *all* the ways (permutations) of selecting 20. In the office, we have 20 options for picking the first person, for the second $ 20-1 = 19 $, we do this until it is only  $ 20-19 = 1 $ person left. To find all the ways of doing this, we *multiply* the series together, arriving at the <strong>factorial</strong> formula:

$n! = n(n-1)(n-2)\cdot \cdot \cdot 2 \cdot 1 $, in our case $20! = 20 \cdot 19 \cdot \cdot \cdot 2 \cdot 1 $

Since we count choices by multiplying number of elements, we *divide* to cancel what we don't count. For counting ways of selecting $k$ elements from $n$, we disregard the rest, namely $(n-k)$ elements. To find the $k$ permutations of $n$, we cancel $(n-k)!$ and get the expression $ n!/(n-k)!$

However, these are the k-permutations of n. To disregard order, we want the *unique* permutations, namely the combinations. Requiring that the $k$ we pick are unique, is again done by reducing (dividing) by the $k!$ ways of selecting $k$ elements:

### $\frac{n!/(n-k)!}{k!} = \binom{n}{k} = \frac{n!}{k!(n-k)!}$

This is called the <strong>binomial coefficient</strong>, the number of k-combinations of $n$ sampled without repetition.

Now, let's implement it in code:

In [19]:
def bnf(n,k): # binomial coefficient
    return int(math.factorial(n) / (math.factorial(k) * math.factorial(n-k)))
bnf(20,2) # try it

190

Finally, we can count the number of 2-combinations of 20 by:
$\binom{20}{2} = \frac{20!}{(20-2)!2!} = \frac{20!}{18!2!} = 190 $

Now that we know how to count selections, the rest is easy! Assume a male population of $N = 2\,500\,000 $ and special elements $ K = 700 $ (www.ssb.no/navn)

We want the probability of getting $k=2$ special elements if we draw $n=20$ from population $N$, where $ K $ are special elements $ k $. As always:

### $ \text{probability} = \frac{\text{valid selections}}{\text{all selections}} $

Let the subsets of $N$ be $[N]$ = {1,...,N}, and count how many of these subsets of size $K$ that have a valid selection, namely draw $n = 20$, with $k=2$ being special: $ {n \choose k} {N - n\choose K - k} $.

Since we draw at random from $N$, we can take the ratio of the number of valid selections and all possible selections of size $K$ of $N$. By now we know this is $N \choose K $, arriving at our final formula:

### $ P(k) = \frac{{n \choose k} {N - n\choose K - k}}{{N \choose K}} $

Behold, the <strong>hypergeometric formula</strong>!


In [31]:
prob=(math.comb(20,2)*math.comb(2500000-20, 700-2))/math.comb(2500000,700)
print('The chance by using binomial coefficients: 1/'+str(round(1/prob)))

from scipy.stats import hypergeom  # The fastest is to use this:
prob = hypergeom(2500000, 20, 700).pmf(2)
print('The chance by using built-in hypergeom: 1/' + str(round(1/prob)))

The chance by using binomial coefficients: 1/67567
The chance by using built-in hypergeom: 1/67567


### So the hypergeometric distribution with our example is:

### $ P(k=2) = \frac{{20 \choose 2} {2\,500\,000 - 20\choose 700 - 2}}{{2\,500\,000 \choose 700}} = \frac{1}{67\,567}$, in line with our simulation!

### A final comment: since <strong>more</strong> than 2 would also be confusing in the office, the probability is actually the cumulative:

In [32]:
prob1 = sum(hypergeom(2500000, 20, 700).pmf(range(2,20)))  # 2,3,...,20 (sum of density)
prob2 = 1-hypergeom(2500000, 20, 700).cdf(1)  # Or 1 - P(k < 2) = 1 - P(0) + P(1)
print('The chance of 2 or more is 1/' + str(round(1/prob2)))

The chance of 2 or more is 1/67424


### Let's check the number of occurences in our 10 million runs:

In [33]:
print(Counter(num_specials))

Counter({0: 9943894, 1: 55963, 2: 143})


### 3 or more did not occur in the simulation...

# Indeed, very <strong>hypergeometric!</strong>

The code can be found at: https://github.com/aannestad/quirks/blob/main/random_seating.ipynb